In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('1_Churn_Modelling(in).csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
df.drop('RowNumber',axis='columns',inplace=True)

In [ ]:
df.drop('CustomerId',axis='columns',inplace=True)

In [ ]:
df.drop('Surname',axis='columns',inplace=True)

In [ ]:
for i in df:
  print(f'{i} : {df[i].unique()}')

CreditScore : [619 608 502 699 850 645 822 376 501 684 528 497 476 549 635 616 653 587
 726 732 636 510 669 846 577 756 571 574 411 591 533 553 520 722 475 490
 804 582 472 465 556 834 660 776 829 637 550 698 585 788 655 601 656 725
 511 614 742 687 555 603 751 581 735 661 675 738 813 657 604 519 664 678
 757 416 665 777 543 506 493 652 750 729 646 647 808 524 769 730 515 773
 814 710 413 623 670 622 785 605 479 685 538 562 721 628 668 828 674 625
 432 770 758 795 686 789 589 461 584 579 663 682 793 691 485 650 754 535
 716 539 706 586 631 717 800 683 704 615 667 484 480 578 512 606 597 778
 514 525 715 580 807 521 759 516 711 618 643 671 689 620 676 572 695 592
 567 694 547 594 673 610 767 763 712 703 662 659 523 772 545 634 739 771
 681 544 696 766 727 693 557 531 498 651 791 733 811 707 714 782 775 799
 602 744 588 747 583 627 731 629 438 642 806 474 559 429 680 749 734 644
 626 649 805 718 840 630 654 762 568 613 522 737 648 443 640 540 460 593
 801 611 802 745 483 690 492 709 705 

In [ ]:
df = pd.get_dummies(data=df,columns=['Geography'],dtype=int)

In [ ]:
df['Gender'].replace({'Female':0,'Male':1},inplace=True)

In [ ]:
df.dtypes

,0
CreditScore,int64
Gender,int64
Age,int64
Tenure,int64
Balance,float64
NumOfProducts,int64
HasCrCard,int64
IsActiveMember,int64
EstimatedSalary,float64
Exited,int64


In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df.drop(['Exited'], axis=1), df['Exited'], test_size=0.2,random_state=42)

In [ ]:
x_train.shape

(8000, 12)

In [ ]:
import keras.layers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU,PReLU,ELU
from keras.layers import Dropout

model = Sequential()
model.add(Dense(units=6,kernel_initializer='he_uniform',activation='relu',input_dim=12))
model.add(Dense(units=6,kernel_initializer='he_uniform',activation='relu'))
model.add(Dense(units=1,kernel_initializer='glorot_uniform',activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# model.fit(x_train,y_train,validation_split=0.33,batch_size=10,epochs=10)


In [ ]:
from keras.callbacks import EarlyStopping,ModelCheckpoint
model2 = Sequential()
model2.add(Dense(units=16, input_shape=(11,), kernel_initializer='random_uniform', activation='relu'))
model2.add(Dense(units=32, kernel_initializer='he_uniform', activation='relu'))
model2.add(Dense(units=64, kernel_initializer='he_uniform', activation='relu'))
model2.add(Dense(units=1, kernel_initializer='he_uniform'))



In [ ]:
model2.compile(loss='mean_squared_error',optimizer='adam',metrics=['mse'])

In [ ]:
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

In [ ]:
checkpoint = ModelCheckpoint(
    'best_model.keras', #saving the model
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

In [ ]:
model.fit(x_train,y_train,epochs=100,batch_size=20,validation_data=(x_test,y_test),callbacks=[early_stop,checkpoint])


Epoch 1/100
374/400 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2987 - loss: 26800.9355
Epoch 1: val_loss improved from inf to 132.94705, saving model to best_model.keras
400/400 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.3093 - loss: 25731.1211 - val_accuracy: 0.6080 - val_loss: 132.9471
Epoch 2/100
391/400 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6997 - loss: 57.3036
Epoch 2: val_loss improved from 132.94705 to 8.59818, saving model to best_model.keras
400/400 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.6994 - loss: 56.7164 - val_accuracy: 0.2685 - val_loss: 8.5982
Epoch 3/100
400/400 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6638 - loss: 34.2392
Epoch 3: val_loss did not improve from 8.59818
400/400 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6638 - loss: 34.2285 - val_accuracy: 0.1970 - val_loss: 20.3226
Epoch 4/100
396/400 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6706 - loss: 20.2515
Epoch 4: val_loss improved from 8.59818 to 3.29181, saving model to 

In [ ]:
model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_19 (Dense)                │ (None, 6)              │            78 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 6)              │            42 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 1)              │             7 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 383 (1.50 KB)

 Trainable params: 127 (508.00 B)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 256 (1.00 KB)

In [ ]:
model3 = Sequential()
model3.add(Dense(units=16, input_shape=(11,), kernel_initializer='random_uniform', activation='relu'))
model3.add(Dense(units=16, input_shape=(11,), kernel_initializer='glorot_uniform', activation='tanh'))
model3.add(Dense(units=32, kernel_initializer='lecun_normal', activation='selu'))
model3.add(Dense(units=64, kernel_initializer='he_normal', activation='elu'))
model3.add(Dense(units=1, kernel_initializer='glorot_normal', activation='sigmoid'))



In [ ]:
model3.compile(loss='mean_squared_error',optimizer='adam',metrics=['mse'])

In [ ]:
early_stop1 = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

1. monitor='val_loss'
What it does: This tells Keras to watch the validation loss during training.

Purpose: When the validation loss stops improving, we want to stop training to avoid overfitting.

You can change this to 'val_accuracy', 'loss', or any other metric being tracked.

2. patience=10
What it does: The training will continue for 10 more epochs after the last improvement in val_loss.

Purpose: Gives the model some time to improve before stopping early. Avoids quitting too soon due to temporary plateaus or noise.

Higher value = more tolerance for fluctuations.

3. restore_best_weights=True
What it does: After stopping early, the model will restore the weights from the epoch where val_loss was best.

Purpose: Ensures the final model is the one that performed best on validation data, not just the last one trained.

Crucial for getting the best generalization performance.



In [ ]:
checkpoint1 = ModelCheckpoint(
    'best_model1.keras', #saving the model
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

In [ ]:
model.fit(x_train,y_train,epochs=100,batch_size=20,validation_data=(x_test,y_test),callbacks=[early_stop1,checkpoint1])

Epoch 1/100
392/400 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6788 - loss: 23.1966
Epoch 1: val_loss improved from inf to 28.94807, saving model to best_model1.keras
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6786 - loss: 23.2000 - val_accuracy: 0.7995 - val_loss: 28.9481
Epoch 2/100
390/400 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6709 - loss: 20.7256
Epoch 2: val_loss improved from 28.94807 to 8.54221, saving model to best_model1.keras
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6708 - loss: 20.7605 - val_accuracy: 0.6155 - val_loss: 8.5422
Epoch 3/100
384/400 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6753 - loss: 17.9896
Epoch 3: val_loss did not improve from 8.54221
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6752 - loss: 18.0682 - val_accuracy: 0.1965 - val_loss: 44.2315
Epoch 4/100
387/400 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6537 - loss: 22.1330
Epoch 4: val_loss did not improve from 8.54221
400/400 ━━━━━━━━━━━━━━━━━━━━

4. verbose=1
What it does: Enables informational messages when early stopping occurs.

Output: You’ll see a message like:

In [ ]:
m4 = Sequential()
m4.add(Dense(units=64,input_dim=12,kernel_initializer='he_uniform',activation='relu'))
m4.add(Dense(units=32,kernel_initializer='he_uniform',activation='relu'))
m4.add(Dense(units=16,kernel_initializer='random_uniform',activation='relu'))
m4.add(Dense(units=8,kernel_initializer ='he_uniform',activation='sigmoid'))
m4.add(Dense(units=4,kernel_initializer ='he_uniform',activation='sigmoid'))
# m4.add(Dense(units=4,kernel_initializer ='ranndom_uniform',activation='relu'))
m4.add(Dense(units=1,activation='sigmoid'))

In [ ]:
m4.compile(loss='mean_squared_error',optimizer='adam',metrics=['mse'])

In [ ]:
m4.fit(x_train,y_train,epochs=100,batch_size=20,validation_data=(x_test,y_test),callbacks=[early_stop1,checkpoint1])

Epoch 1/100
389/400 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2036 - mse: 0.2036
Epoch 1: val_loss improved from inf to 0.15903, saving model to best_model1.keras
400/400 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.2030 - mse: 0.2030 - val_loss: 0.1590 - val_mse: 0.1590
Epoch 2/100
393/400 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1625 - mse: 0.1625
Epoch 2: val_loss improved from 0.15903 to 0.15801, saving model to best_model1.keras
400/400 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.1625 - mse: 0.1625 - val_loss: 0.1580 - val_mse: 0.1580
Epoch 3/100
392/400 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1588 - mse: 0.1588
Epoch 3: val_loss did not improve from 0.15801
400/400 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.1589 - mse: 0.1589 - val_loss: 0.1580 - val_mse: 0.1580
Epoch 4/100
395/400 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1649 - mse: 0.1649
Epoch 4: val_loss improved from 0.15801 to 0.15791, saving model to best_model1.keras
400/400 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0

Different Kernal_initializers
1. random_uniform  
2. random_normal  
3. zeros  
4. ones  
5. he_uniform  
6. he_normal  
7. glorot_uniform  
8. glorot_normal  
9. lecun_uniform  
10. lecun_normal  
11. orthogonal  
12. identity  
13. constant


Different Activation Functions

1. relu  
2. sigmoid  
3. tanh  
4. softmax  
5. linear  
6. elu  
7. selu  
8. softplus  
9. softsign  
10. swish  
11. gelu


In [72]:
m5=Sequential()
m5.add(Dense(units=16,kernel_initializer='random_uniform',activation='relu',input_dim=12))
m5.add(Dense(units=64,kernel_initializer='he_uniform',activation='relu'))
m5.add(Dense(units=128,kernel_initializer='he_uniform',activation='relu'))
m5.add(Dense(units=256,kernel_initializer='he_uniform',activation='relu'))
m5.add(Dense(units=512,kernel_initializer='he_uniform',activation='relu'))
m5.add(Dense(units=1024,kernel_initializer='he_uniform',activation='relu'))
m5.add(Dense(units=128,kernel_initializer='he_uniform',activation='relu'))
m5.add(Dense(units=64,kernel_initializer='he_uniform',activation='relu'))
m5.add(Dense(units=16,kernel_initializer='he_uniform',activation='relu'))
m5.add(Dense(units=1,kernel_initializer='he_uniform',activation='sigmoid'))
m5.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])
m5.fit(x_train,y_train,validation_split=0.35,batch_size=40,epochs=100)



In [ ]:
from keras.callbacks import EarlyStopping,ModelCheckpoint
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    mode='min',
    restore_best_weights=True,
    verbose=1
    )

checkpoint = ModelCheckpoint(
    'best_model_my_10_layes.keras', #saving the model
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

In [ ]:
m5.fit(x_train,y_train,epochs=100,batch_size=20,validation_split=0.2,validation_data=(x_test,y_test),callbacks=[early_stop,checkpoint])

Epoch 1/100
400/400 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.2060 - mse: 0.2060
Epoch 1: val_loss improved from inf to 0.19650, saving model to best_model_my_10_layes.keras
400/400 ━━━━━━━━━━━━━━━━━━━━ 8s 18ms/step - loss: 0.2060 - mse: 0.2060 - val_loss: 0.1965 - val_mse: 0.1965
Epoch 2/100
399/400 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.2017 - mse: 0.2017
Epoch 2: val_loss did not improve from 0.19650
400/400 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - loss: 0.2017 - mse: 0.2017 - val_loss: 0.1965 - val_mse: 0.1965
Epoch 3/100
400/400 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.2034 - mse: 0.2034
Epoch 3: val_loss did not improve from 0.19650
400/400 ━━━━━━━━━━━━━━━━━━━━ 10s 14ms/step - loss: 0.2034 - mse: 0.2034 - val_loss: 0.1965 - val_mse: 0.1965
Epoch 4/100
399/400 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.2083 - mse: 0.2083
Epoch 4: val_loss did not improve from 0.19650
400/400 ━━━━━━━━━━━━━━━━━━━━ 12s 19ms/step - loss: 0.2083 - mse: 0.2083 - val_loss: 0.1965 - val_mse: 0.1965


Adding Drop Out LAyers

Drop Out Layer
it works on
1) co-Adaptation
2) over-reliance

It is used to overcome overfitting

<30% dropout is good

In [73]:
m6=Sequential()
m6.add(Dense(units=16,kernel_initializer='random_uniform',activation='relu',input_dim=12))
m6.add(Dense(units=64,kernel_initializer='he_uniform',activation='relu'))
m6.add(Dropout(0.2))
m6.add(Dense(units=128,kernel_initializer='he_uniform',activation='relu'))
m6.add(Dense(units=256,kernel_initializer='he_uniform',activation='relu'))
m6.add(Dropout(0.4))
m6.add(Dense(units=512,kernel_initializer='he_uniform',activation='relu'))
m6.add(Dense(units=1024,kernel_initializer='he_uniform',activation='relu'))
m6.add(Dropout(0.6))
m6.add(Dense(units=128,kernel_initializer='he_uniform',activation='relu'))
m6.add(Dense(units=64,kernel_initializer='he_uniform',activation='relu'))
m6.add(Dropout(0.15))
m6.add(Dense(units=16,kernel_initializer='he_uniform',activation='relu'))
m6.add(Dense(units=1,kernel_initializer='he_uniform',activation='sigmoid'))
m6.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])




In [74]:
from keras.callbacks import EarlyStopping,ModelCheckpoint
early_stop_for_dropout = EarlyStopping(
    monitor='val_loss',
    patience=10,
    mode='min',
    restore_best_weights=True,
    verbose=1
    )

checkpoint_dropout = ModelCheckpoint(
    'with_dropout_layer.keras', #saving the model
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

In [75]:
m6.fit(x_train,y_train,epochs=100,batch_size=20,validation_split=0.2,validation_data=(x_test,y_test),callbacks=[early_stop,checkpoint])

Epoch 1/100
398/400 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.2607 - mse: 0.2607
Epoch 1: val_loss improved from inf to 0.19650, saving model to best_model_my_10_layes.keras
400/400 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - loss: 0.2604 - mse: 0.2604 - val_loss: 0.1965 - val_mse: 0.1965
Epoch 2/100
397/400 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.2061 - mse: 0.2061
Epoch 2: val_loss did not improve from 0.19650
400/400 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - loss: 0.2061 - mse: 0.2061 - val_loss: 0.1965 - val_mse: 0.1965
Epoch 3/100
400/400 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.2022 - mse: 0.2022
Epoch 3: val_loss did not improve from 0.19650
400/400 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - loss: 0.2022 - mse: 0.2022 - val_loss: 0.1965 - val_mse: 0.1965
Epoch 4/100
397/400 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.2068 - mse: 0.2068
Epoch 4: val_loss did not improve from 0.19650
400/400 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - loss: 0.2067 - mse: 0.2067 - val_loss: 0.1965 - val_mse: 0.1965


In [76]:
m6=Sequential()
m6.add(Dense(units=16,kernel_initializer='random_uniform',activation='relu',input_dim=12))
m6.add(Dense(units=64,kernel_initializer='he_uniform',activation='relu'))
m6.add(Dropout(0.2))
m6.add(Dense(units=128,kernel_initializer='he_uniform',activation='relu'))
m6.add(Dropout(0.25))
m6.add(Dense(units=256,kernel_initializer='he_uniform',activation='relu'))
m6.add(Dropout(0.3))
m6.add(Dense(units=512,kernel_initializer='he_uniform',activation='relu'))
m6.add(Dropout(0.4))
m6.add(Dense(units=1024,kernel_initializer='he_uniform',activation='relu'))
m6.add(Dropout(0.35))
m6.add(Dense(units=128,kernel_initializer='he_uniform',activation='relu'))
m6.add(Dropout(0.4))
m6.add(Dense(units=64,kernel_initializer='he_uniform',activation='relu'))
m6.add(Dropout(0.15))
m6.add(Dense(units=16,kernel_initializer='he_uniform',activation='relu'))
m6.add(Dense(units=1,kernel_initializer='he_uniform',activation='sigmoid'))
m6.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])

In [77]:
m6.fit(x_train,y_train,epochs=100,batch_size=20,validation_split=0.2,validation_data=(x_test,y_test),callbacks=[early_stop,checkpoint])

Epoch 1/100
397/400 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.4561 - mse: 0.4561
Epoch 1: val_loss did not improve from 0.19650
400/400 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - loss: 0.4573 - mse: 0.4573 - val_loss: 0.8035 - val_mse: 0.8035
Epoch 2/100
399/400 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.6630 - mse: 0.6630
Epoch 2: val_loss did not improve from 0.19650
400/400 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - loss: 0.6622 - mse: 0.6622 - val_loss: 0.1965 - val_mse: 0.1965
Epoch 3/100
398/400 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.2252 - mse: 0.2252
Epoch 3: val_loss did not improve from 0.19650
400/400 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - loss: 0.2252 - mse: 0.2252 - val_loss: 0.1965 - val_mse: 0.1965
Epoch 4/100
399/400 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.2135 - mse: 0.2135
Epoch 4: val_loss did not improve from 0.19650
400/400 ━━━━━━━━━━━━━━━━━━━━ 10s 17ms/step - loss: 0.2135 - mse: 0.2135 - val_loss: 0.1965 - val_mse: 0.1965
Epoch 5/100
399/400 ━━━━━━━━━━━━━━━━━━━━ 0s 

In [81]:
m7 = Sequential()
m7.add(Dense(units=16, input_shape=(12,), kernel_initializer='random_uniform', activation='relu'))
m7.add(Dropout(0.2))
m7.add(Dense(units=32, kernel_initializer='he_uniform', activation='relu'))
m7.add(Dropout(0.2))
m7.add(Dense(units=64, kernel_initializer='he_uniform', activation='relu'))
m7.add(Dropout(0.2))
m7.add(Dense(units=1, kernel_initializer='he_uniform'))
m7.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])

In [82]:
early_stop_for_dropout_4l = EarlyStopping(
    monitor='val_loss',
    patience=2,
    mode='min',
    restore_best_weights=True,
    verbose=1
    )

checkpoint_dropout_4l = ModelCheckpoint(
    'with_dropout_layer_4l.keras', #saving the model
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

In [83]:
m7.fit(x_train,y_train,epochs=100,batch_size=20,validation_split=0.2,validation_data=(x_test,y_test),callbacks=[early_stop_for_dropout_4l,checkpoint_dropout_4l])

Epoch 1/100
381/400 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2394084.0000 - mse: 2394084.0000
Epoch 1: val_loss improved from inf to 6.33618, saving model to with_dropout_layer_4l.keras
400/400 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 2308353.2500 - mse: 2308353.2500 - val_loss: 6.3362 - val_mse: 6.3362
Epoch 2/100
400/400 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2342.6755 - mse: 2342.6755
Epoch 2: val_loss improved from 6.33618 to 0.32648, saving model to with_dropout_layer_4l.keras
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 2339.6621 - mse: 2339.6621 - val_loss: 0.3265 - val_mse: 0.3265
Epoch 3/100
386/400 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 165.9343 - mse: 165.9343
Epoch 3: val_loss improved from 0.32648 to 0.25734, saving model to with_dropout_layer_4l.keras
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 164.5197 - mse: 164.5197 - val_loss: 0.2573 - val_mse: 0.2573
Epoch 4/100
396/400 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 37.9334 - mse: 37.9334
Epoch 4: val_loss imp